# IMPORT PACKAGES

In [ ]:
import numpy as np
import glob
from igraph import plot
from igraph import drawing
#import cairo
import networkx as nx
import matplotlib.pyplot as plt
from igraph import Graph
from numpy import matrix
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
import numpy as np
import glob
import seaborn as sns
from kerastuner.tuners import RandomSearch
import matplotlib.pyplot as plt
from numpy import matrix
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from pandas import DataFrame
from sklearn import datasets
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import kerastuner as kt
from pandas import DataFrame
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras import layers
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import callbacks
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D
)
from kerastuner.tuners import Hyperband

from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
import kerastuner as kt
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from kerastuner import HyperModel
from tensorflow.python.keras.layers import Input, Dense, LSTM
from sklearn import covariance, preprocessing

# READ FILES

In [ ]:

def pixels_from_path(file_path):
    im = open(file_path)
    test = pd.read_csv(im,sep=",", index_col=False,header=None)
    #im = im.resize(IMG_SIZE)
    np_im = np.array(test)
    #matrix of pixel RGB values
    return np_im

print("Dystonia")
dystonia = np.asarray([pixels_from_path(n) for n in glob.glob(r'INSERT MATRICES DYSTONIA FOLDER PATH')])
print(dystonia)
print(np.shape(dystonia))
print("Normal")
normal = np.asarray([pixels_from_path(n) for n in glob.glob(r'INSERT MATRICES NORMAL FOLDER PATH')])
print(normal)
print(np.shape(normal))



# PREPARING TRAIN SET

In [ ]:
x_train = np.concatenate([normal ,dystonia])
print(np.shape(x_train))
myScaler = preprocessing.StandardScaler()
x_train=myScaler.fit_transform(i)
x=np.nan_to_num(x_train) 


# PREPARING LABELS

In [ ]:
disease = [1]*len(dystonia)
no_disease = [0]*len(normal)
labels = disease + no_disease 
print(len(labels))## the labels of the graphs
y=np.nan_to_num(labels)

# SPLIT TRAIN AND TEST

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25)

# DEFINE CNN MODEL

In [ ]:

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
class LSTM_model(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()



        model.add(LSTM(units=70, # dimensionality of the output space 
               dropout=0.4, # Fraction of the units to drop (inputs)
               recurrent_dropout=0.15, # Fraction of the units to drop (recurent state)
               return_sequences=True, # return the last state in addition to the output
               input_shape= self.input_shape ))

        model.add(LSTM(units=60,  
               dropout=0.4, 
               recurrent_dropout=0.15, 
               return_sequences=True))

        model.add(LSTM(units=50,  
               dropout=0.4, 
               recurrent_dropout=0.15, 
               return_sequences=True))

        model.add(LSTM(units=40,  
               dropout=0.4, 
               recurrent_dropout=0.15, 
               return_sequences=False))


        model.add(Dense(self.num_classes, activation='softmax'))
    

  
        return model

# CALL THE MODEL

In [ ]:
#from keras.utils import to_categorical 
from keras.utils import np_utils
SEED = 1
NUM_CLASSES = 2  #
HYPERBAND_MAX_EPOCHS = 10
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 2
INPUT_SHAPE = (32, 32, 1) 
list_accuracy= []
list_precision=[]
list_recall=[]
list_auc=[]
X_train, X_test, Y_train, Y_test = train_test_split(x, y, stratify=y, test_size=0.25)
skfold = StratifiedKFold(n_splits=5)
for train_index, test_index in skfold.split(X_train,Y_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X_train[train_index], X_train[test_index]
    y_train, y_test = Y_train[train_index], Y_train[test_index]
    print(np.shape(x_train))
    
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)

    hypermodel = LSTM_model(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
    tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
   
     
    overwrite=True    

    )
    tuner.search(x_train, y_train, epochs=100,  validation_data=(x_test,y_test), verbose=0)
    best_model_rs = tuner.get_best_models(num_models=1)[0]
    history_rs=best_model_rs.fit(x_train,y_train, epochs=1000, validation_data=(x_test,y_test))
    print(best_model_rs.evaluate(x_train, y_train))
    list_accuracy.append(history_rs.history['accuracy'])
    list_precision.append(history_rs.history['precision'])
    list_recall.append(history_rs.history['recall'])
    list_auc.append(history_rs.history['auc'])


#  RESULTS MODEL


In [ ]:
best_model_rs.evaluate(x_train, y_train)

print('Accuracy', np.mean(list_accuracy))
print('AUC',np.mean(list_auc))
print('Recall',np.mean(list_recall))
print('Precision',np.mean(list_precision))


print(np.shape(X_test))
print(np.shape(x_test))
Y_test1=np_utils.to_categorical(Y_test)
best_model_rs.evaluate(X_test, Y_test1)


# ROC CURVE

In [ ]:

sns.set_style('whitegrid')
y_score= best_model_rs.predict(X_test)
# Plot linewidth.
lw = 2
n_classes=2
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test1[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Y_test1[:, i], Y_test1[:, i])
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(figsize=(8,5))
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='slategray', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['purple', 'lightseagreen'])
list_names=['HC','ULD']
for i, color, name in zip(range(n_classes), colors,list_names):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
        label="ROC curve of class "+ name + " (area = {1:0.2f})".format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--',color='#cb416b', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.annotate(' Random Guess',(.5,.48),color='#cb416b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Receiver Operating Characteristic for Suport Vector Machine')
plt.legend(loc="lower right")
plt.savefig("dystonia-lstm-tunned-roc.pdf",dpi=300, bbox_inches = "tight")
plt.show()


# CONFUSION MATRIX

In [ ]:
cm = metrics.confusion_matrix(y_score, Y_test)
cm_sum = np.sum(cm, axis=1, keepdims=True)
cm_perc = cm / cm_sum.astype(float) * 100
annot = np.empty_like(cm).astype(str)
nrows, ncols = cm.shape
for i in range(nrows):
    for j in range(ncols):
        c = cm[i, j]
        p = cm_perc[i, j]
        if i == j:
            s = cm_sum[i]
            annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
        elif c == 0:
            annot[i, j] = ''
        else:
            annot[i, j] = '%.1f%%\n%d' % (p, c)
cm = pd.DataFrame(cm)
print(cm)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
sns.heatmap(cm, annot=annot, fmt='',cmap='rocket_r')

plt.savefig("dystonia-LSTM-tunned-confusion-matrix.pdf",dpi=2000, bbox_inches = "tight",xticklabels=list_names, yticklabels=list_names)